In [1]:
# Importando bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report
# r2
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_error

#Importando modelos de machine learning
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor

In [2]:
# Importando base de dados que contem o header
base = pd.read_csv('/home/apo-note/Documents/GitHub/Classificacao_MHI_BIC/Codigos-Desenvolvimento/Datasets/df_soc_whoqol_reg.csv')
# Dividindo entre X e y, pelo tamanho da base
X = base.drop('Nivel_MHI', axis=1)
X = X.drop('MHI_Score', axis=1)
y = base['MHI_Score'].apply(lambda x: x/100)

df_Niveis = base['Nivel_MHI']

In [3]:
y.describe()

count    872.000000
mean       1.385585
std        0.335148
min        0.470000
25%        1.170000
50%        1.390000
75%        1.640000
max        2.240000
Name: MHI_Score, dtype: float64

In [4]:
# Dividindo a base entre treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [43]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
# Linear Regression
from sklearn.linear_model import LinearRegression
# Decision Tree
from sklearn.tree import DecisionTreeRegressor
# K-Nearest Neighbors
from sklearn.neighbors import KNeighborsRegressor
import math
import time

def test_and_tune_regression_models(x_train, y_train, x_test, y_test, filename='regression_results.csv'):
  """
  Tests various regression models, displays results, performs Random Search
  for hyperparameter tuning, and presents the tuned results.

  Args:
      x_train (pandas.DataFrame): Training data features.
      y_train (pandas.Series): Training data target variable.
      x_test (pandas.DataFrame): Testing data features.
      y_test (pandas.Series): Testing data target variable.
      filename (str): Name of the CSV file to save results to.

  Returns:
      None
  
  Outputs:
      CSV file: A CSV file containing the results of the regression models.
  """

  # Define models and their parameter grids for random search
  models = {
      'Linear Regression': {
          'model': LinearRegression(),
          'param_grid': {}  # Linear Regression has no hyperparameters to tune
      },
      'Decision Tree': {
          'model': DecisionTreeRegressor(),
          'param_grid': {
            #   'max_depth': range(2, 10),
            #   'min_samples_split': range(2, 20),
            #   'min_samples_leaf': range(1, 10)
                'criterion': ['absolute_error', 'squared_error', 'friedman_mse', 'poisson'],
                'splitter': ['best', 'random'],
                'max_depth': np.unique(np.linspace(math.log2(len(np.unique(y))), len(X.columns) // 2, 20, dtype=int)),
                'min_samples_split': np.linspace(0.01, 1, 20),
                'min_samples_leaf': np.linspace(0.01, 0.5, 20)
          }
      },
      'Random Forest': {
          'model': RandomForestRegressor(n_jobs=-1),
          'param_grid': {
            #   'n_estimators': range(100, 1000, 100),
            #   'max_depth': range(2, 10),
            #   'min_samples_split': range(2, 20),
            #   'min_samples_leaf': range(1, 10)
                # Vetor linear inteiro, entre log2(numero_de_classes) e numero_de_features/2
                'max_depth': np.unique(np.linspace(math.log2(len(np.unique(y))), len(X.columns) // 2, 20, dtype=int)),
                # Vetor linear inteiro, de ln(numero de objetos no conjunto) -40 até ln(numero de objetos no conjunto) + 45
                'n_estimators': np.unique(np.maximum(np.linspace(math.log(len(X)) - 40, math.log(len(X)) + 45, 20, dtype=int),1)),
                # Vetor linear inteiro, numero features * 0.3 até numero features * 0.8
                'max_features': np.unique(np.linspace(int(len(X.columns) * 0.3), int(len(X.columns) * 0.8), 20, dtype=int)),
                'criterion': ['absolute_error', 'squared_error', 'friedman_mse', 'poisson'],
                'bootstrap': [True, False]
          }
      },
      'XGBoost': {
          'model': XGBRegressor(n_jobs=-1),
          'param_grid': {
              'n_estimators': range(100, 1000, 100),
              'learning_rate': [0.01, 0.1, 1],
              'max_depth': range(2, 10),
              'min_child_weight': range(1, 10),
              'gamma': [0, 0.1, 1]
          }
      },
      'K-Nearest Neighbors': {
          'model': KNeighborsRegressor(n_jobs=-1),
          'param_grid': {
                'n_neighbors': np.linspace(1, 30, 15, dtype=int),
                'weights': ['uniform', 'distance'],
                'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                'p': [1, 2, 3, 4],
                'leaf_size': np.linspace(1, 100, 20, dtype=int)
          }
      },
      'Multi-Layer Perceptron (Neural Network)': {
          'model': MLPRegressor(),
          'param_grid': {
            #   'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 100)],
                'activation': ['identity', 'logistic', 'tanh', 'relu'],
                'solver': ['lbfgs', 'sgd', 'adam'],
                'hidden_layer_sizes': [(neurons,) * layers for layers in range(1, 4) for neurons in range(len(X.columns) * 5, len(X.columns) * 10 + 1)],
                'learning_rate_init': np.linspace(0.0001, 1, 10),
                'momentum': np.linspace(0.1, 0.9, 10),
                'max_iter': np.linspace(400, 4000, 20, dtype=int)
          }
      },
      'Support Vector Regression (SVR)': {
          'model': SVR(),
          'param_grid': {
            'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
            'C': [0.00001, 0.0001, 0.001,  0.01, 0.1, 1, 10, 100, 1000, 10000],
            # 'degree': [3, 5, 8],
            'coef0': [0.01, 10, 0.5],
            'gamma': ['auto', 'scale']
            }
      }

}

  # Prepare empty DataFrame to store results
  results = pd.DataFrame(columns=['Modelo', 'Pre-Tuning MSE', 'Pre-Tuning RMSE', 'Pre-Tuning R2', 'Pre-Tuning R2 Ajustado', 'Pos-Tuning MSE', 'Pos-Tuning RMSE', 'Pos-Tuning R2', 'Pos-Tuning R2 Ajustado' 'Melhores Parametros'])

  # Test each model, store results, and print
  for model_name, model_info in models.items():
    model = model_info['model']
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    # Update results DataFrame
    results = pd.concat([results, pd.DataFrame({
        'Modelo': [model_name],
        'Pre-Tuning MSE': [mse],
        'Pre-Tuning RMSE': [rmse],
        'Pre-Tuning R2': [r2],
        'Pre-Tuning R2 Ajustado': [1 - (1 - r2) * (len(y_test) - 1) / (len(y_test) - len(X.columns) - 1)], # Adjusted R2 = 1 - (1 - R2) * (n - 1) / (n - p - 1)
        'Pos-Tuning MSE': [None],
        'Pos-Tuning RMSE': [None],
        'Pos-Tuning R2': [None],
        'Pos-Tuning R2 Ajustado': [None],
        'Melhores Parametros': [None]
    })], ignore_index=True)

    print(f"\n\n**{model_name} Resultados (Antes do Tuning):**")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R2: {r2:.4f}")
    print(f"R2 Ajustado: {1 - (1 - r2) * (len(y_test) - 1) / (len(y_test) - len(X.columns) - 1):.4f}")

    print(f'\n_Realizando Tuning do Modelo: {model_name}_')
    agora = time.time()
    param_grid = model_info['param_grid']
    random_search = RandomizedSearchCV(model, param_grid, n_iter=25, cv=2, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)
    if model_name == 'Support Vector Regression (SVR)': # Use only a subset of the data for SVR to speed up tuning
        print("Usando subset")
        x_train_subset, _, y_train_subset, _ = train_test_split(x_train, y_train, train_size=0.3, shuffle=True)
        random_search.fit(x_train_subset, y_train_subset)
    else:
        random_search.fit(x_train, y_train)
    best_model = random_search.best_estimator_
    y_pred = best_model.predict(x_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    r2_ajustado = 1 - (1 - r2) * (len(y_test) - 1) / (len(y_test) - len(X.columns) - 1)
    depois = time.time()

    print(f"Tempo de execução: {depois - agora:.2f} segundos.")

    # Update results DataFrame
    results.loc[results['Modelo'] == model_name, 'Pos-Tuning MSE'] = mse
    results.loc[results['Modelo'] == model_name, 'Pos-Tuning RMSE'] = rmse
    results.loc[results['Modelo'] == model_name, 'Pos-Tuning R2'] = r2
    results.loc[results['Modelo'] == model_name, 'Pos-Tuning R2 Ajustado'] = r2_ajustado
    results.loc[results['Modelo'] == model_name, 'Melhores Parametros'] = random_search.best_params_

    print(f"**{model_name} Resultados (Depois do Tuning):**")
    print(f"Melhores Parametros: {random_search.best_params_}")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R2: {r2:.4f}")
    print(f"R2 Ajustado: {r2_ajustado:.4f}")

  # Save results to CSV file
  results.to_csv(filename, index=False)

  print(f"\nResultados salvos em: '{filename}'.")

In [45]:
test_and_tune_regression_models(X_train, y_train, X_test, y_test)



**Linear Regression Resultados (Antes do Tuning):**
MSE: 0.0306
RMSE: 0.1750
R2: 0.7262

_Realizando Tuning do Modelo: Linear Regression_
Fitting 2 folds for each of 1 candidates, totalling 2 fits


/tmp/ipykernel_411826/250377636.py:129: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, pd.DataFrame({
/home/apo-note/anaconda3/envs/bic/lib/python3.12/site-packages/sklearn/model_selection/_search.py:318: UserWarning: The total space of parameters 1 is smaller than n_iter=25. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
Tempo de execução: 0.98 segundos.
**Linear Regression Resultados (Depois do Tuning):**
Melhores Parametros: {}
MSE: 0.0306
RMSE: 0.1750
R2: 0.7262


**Decision Tree Resultados (Antes do Tuning):**
MSE: 0.0768
RMSE: 0.2772
R2: 0.3129

_Realizando Tuning do Modelo: Decision Tree_
Fitting 2 folds for each of 25 candidates, totalling 50 fits
[CV] END criterion=poisson, max_depth=34, min_samples_leaf=0.08736842105263157, min_samples_split=0.8436842105263158, splitter=random; total time=   0.0s
[CV] END criterion=poisson, max_depth=25, min_samples_leaf=0.44842105263157894, min_samples_split=0.8436842105263158, splitter=random; total time=   0.0s
[CV] END criterion=poisson, max_depth=25, min_samples_leaf=0.44842105263157894, min_samples_split=0.8436842105263158, splitter=random; total time=   0.0s
[CV] END criterion=poisson, max_dept

/home/apo-note/anaconda3/envs/bic/lib/python3.12/site-packages/sklearn/utils/extmath.py:208: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/home/apo-note/anaconda3/envs/bic/lib/python3.12/site-packages/sklearn/utils/extmath.py:208: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/apo-note/anaconda3/envs/bic/lib/python3.12/site-packages/sklearn/utils/extmath.py:208: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/home/apo-note/anaconda3/envs/bic/lib/python3.12/site-packages/sklearn/utils/extmath.py:208: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/apo-note/anaconda3/envs/bic/lib/python3.12/site-packages/sklearn/utils/extmath.py:208: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/home/apo-note/anaconda3/envs/bic/lib/python3.12/site-packages/sklearn/utils/extmath.py:208: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/apo-note/anaconda3/envs/bic/lib/python3.12/site-package

[CV] END activation=tanh, hidden_layer_sizes=(722, 722), learning_rate_init=0.8889, max_iter=1536, momentum=0.7222222222222222, solver=sgd; total time=   3.3s
[CV] END activation=tanh, hidden_layer_sizes=(722, 722), learning_rate_init=0.8889, max_iter=1536, momentum=0.7222222222222222, solver=sgd; total time=   3.7s
[CV] END activation=tanh, hidden_layer_sizes=(423, 423, 423), learning_rate_init=0.6667000000000001, max_iter=3052, momentum=0.1, solver=adam; total time=   3.5s
[CV] END activation=tanh, hidden_layer_sizes=(423, 423, 423), learning_rate_init=0.6667000000000001, max_iter=3052, momentum=0.1, solver=adam; total time=   3.2s
[CV] END activation=relu, hidden_layer_sizes=(835,), learning_rate_init=0.2223, max_iter=3810, momentum=0.2777777777777778, solver=adam; total time=   6.9s
[CV] END activation=tanh, hidden_layer_sizes=(795,), learning_rate_init=0.2223, max_iter=3052, momentum=0.2777777777777778, solver=sgd; total time=   0.7s
[CV] END activation=tanh, hidden_layer_sizes=(7

...

[CV] END activation=identity, hidden_layer_sizes=(521, 521, 521), learning_rate_init=1.0, max_iter=968, momentum=0.8111111111111111, solver=sgd; total time= 4.1min
[CV] END activation=logistic, hidden_layer_sizes=(519, 519, 519), learning_rate_init=0.33340000000000003, max_iter=2294, momentum=0.5444444444444445, solver=adam; total time= 1.3min


/home/apo-note/anaconda3/envs/bic/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1915) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV] END activation=relu, hidden_layer_sizes=(763, 763, 763), learning_rate_init=0.2223, max_iter=1915, momentum=0.7222222222222222, solver=sgd; total time= 6.9min


/home/apo-note/anaconda3/envs/bic/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1915) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/apo-note/anaconda3/envs/bic/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:547: FitFailedWarning: 
8 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
8 fits failed with the following error:
Traceback (most recent call last):
  File "/home/apo-note/anaconda3/envs/bic/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/apo-n

[CV] END activation=relu, hidden_layer_sizes=(763, 763, 763), learning_rate_init=0.2223, max_iter=1915, momentum=0.7222222222222222, solver=sgd; total time= 7.2min
Tempo de execução: 499.37 segundos.
**Multi-Layer Perceptron (Neural Network) Resultados (Depois do Tuning):**
Melhores Parametros: {'solver': 'lbfgs', 'momentum': 0.3666666666666667, 'max_iter': 2484, 'learning_rate_init': 1.0, 'hidden_layer_sizes': (593, 593, 593), 'activation': 'logistic'}
MSE: 0.0307
RMSE: 0.1753
R2: 0.7252


**Support Vector Regression (SVR) Resultados (Antes do Tuning):**
MSE: 0.0401
RMSE: 0.2002
R2: 0.6417

_Realizando Tuning do Modelo: Support Vector Regression (SVR)_
Usando subset
Fitting 2 folds for each of 25 candidates, totalling 50 fits
[CV] END ....C=10000, coef0=0.01, gamma=auto, kernel=sigmoid; total time=   0.0s
[CV] END .......C=1e-05, coef0=0.5, gamma=scale, kernel=poly; total time=   0.0s
[CV] END .......C=0.001, coef0=0.5, gamma=scale, kernel=poly; total time=   0.0s
[CV] END .......C=0.

In [12]:
import multiprocessing
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
import os
# Função para aplicar SBFS e SFFS com cross validation e salvar tabela com resultado.
def perform_feature_selection(data, target, model, best_model, method, cv):
    n_cpus = multiprocessing.cpu_count()
    # Criando o objeto de seleção de features
    if method == 'SBFS':
        sfs = SFS(model,
                  k_features="best",
                  forward=False,
                  floating=True,
                  verbose=2,
                  scoring='neg_mean_squared_error',
                  cv=cv,
                  n_jobs=n_cpus)
    elif method == 'SFFS':
        sfs = SFS(model,
                  k_features="best",
                  forward=True,
                  floating=True,
                  verbose=2,
                  scoring='neg_mean_squared_error',
                  cv=cv,
                  n_jobs=n_cpus)
    # Aplicando o método de seleção de features
    sfs = sfs.fit(data, target)
    # Salvando o resultado em um dataframe
    df = pd.DataFrame.from_dict(sfs.get_metric_dict()).T

    # Definindo nome do arquivo para salvar as features selecionadas em formato CSV
    if os.path.exists(f"results/{method}_{best_model}.csv"):
        i = 1
        while os.path.exists(f"results/{method}_{best_model}_{i}.csv"):
            i += 1
        file_name_csv = f"results/{method}_{best_model}_{i}.csv"
    else:
        file_name_csv = f"results/{method}_{best_model}.csv"

    # Salvando o dataframe em um arquivo CSV
    df.to_csv(file_name_csv, index=False)

    # Definindo nome do arquivo para salvar as features selecionadas em formato TXT
    if os.path.exists(f"results/{method}_{best_model}.txt"):
        i = 1
        while os.path.exists(f"results/{method}_{best_model}_{i}.txt"):
            i += 1
        file_name_txt = f"results/{method}_{best_model}_{i}.txt"
    else:
        file_name_txt = f"results/{method}_{best_model}.txt"

    # Salvando as features selecionadas e o score em um arquivo TXT
    with open(file_name_txt, 'w') as file:
        file.write(f"Selected Features: {', '.join(sfs.k_feature_names_)}\n")
        file.write(f"Score: {sfs.k_score_}")

    # Plotando o desempenho do modelo para cada combinação de atributos
    fig, ax = plt.subplots()
    ax.plot(range(1, len(df) + 1), df['avg_score'], marker='o')
    ax.set_xlabel('Number of Features')
    ax.set_ylabel('Average Score')
    ax.set_title('Performance of Model with Feature Selection')
    plt.xticks(range(1, len(df) + 1))
    plt.grid(True)

    # Definindo nome do arquivo para salvar o gráfico
    if os.path.exists(f"results/{method}_{best_model}.png"):
        i = 1
        while os.path.exists(f"results/{method}_{best_model}_{i}.png"):
            i += 1
        file_name_plot = f"results/{method}_{best_model}_{i}.png"
    else:
        file_name_plot = f"results/{method}_{best_model}.png"

    # Salvando o gráfico em um arquivo PNG
    plt.savefig(file_name_plot)

    # Retornando as features selecionadas e o score
    return sfs.k_feature_names_, sfs.k_score_

In [15]:
#'solver': 'lbfgs', 'momentum': 0.3666666666666667, 'max_iter': 2484, 'learning_rate_init': 1.0, 'hidden_layer_sizes': (593, 593, 593), 'activation': 'logistic'
# Definindo o modelo de regressão
# Disable warnings
import warnings
warnings.filterwarnings("ignore")

model = MLPRegressor(solver='lbfgs', momentum=0.3666666666666667, max_iter=2484, learning_rate_init=1.0, hidden_layer_sizes=(593, 593, 593), activation='logistic')

# Aplicando o método de seleção de features
features, score = perform_feature_selection(X_train, y_train, model, 'MLPRegressor', 'SFFS', 5)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
/home/apo-note/anaconda3/envs/bic/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(
/home/apo-note/anaconda3/envs/bic/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(
/home/apo-note/anaconda3/envs/bic/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(
/home/apo-note/anaconda3/envs/bic/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `par

AttributeError: SequentialFeatureSelector has not been fitted, yet.

In [11]:
from sklearn.metrics import get_scorer_names
get_scorer_names()

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'matthews_corrcoef',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_negative_likelihood_ratio',
 'neg_root_mean_squared_error',
 'neg_root_mean_squared_log_error',
 'normalized_mutual_info_score',
 'positive_likelihood_ratio',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 '